## IBM data science certificate 

### 1. Import Libraries

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install geopy
from geopy.geocoders import Nominatim ### convert an address into latitude & longitude values
!pip install geocoder
import geocoder
import requests # library to handle request
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize  # transform JSON file into a pandas dataframe

#matplotlib library
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('libraries imported')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Elvis Ma\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py37_0         749 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.8 MB

The following NEW packages will be INSTALLED:

  altair             conda-forge/win-64::altair-3.2.0-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium



==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda




### 2. Web scraping to get data of Dublin neighborhoods

#### let's get the coordinates of Dublin, CA

In [3]:
address='Dublin, CA'
geolocator=Nominatim(user_agent='dublin_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Dublin, CA are {},{}'.format(latitude,longitude))

The geographical coordinate of Dublin, CA are 37.7021521,-121.9357918


#### let's look into the neighborhood data from nextdoor.com

In [4]:
url='https://nextdoor.com/city/dublin--ca/'
dublin_results=requests.get(url).text
soup=BeautifulSoup(dublin_results,'lxml')
soup.find_all('div')

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="#19975d" name="theme-color"/>
<meta content="68022CCBA99514E28D9C825C3B4D4BA6" name="msvalidate.01"/>
<meta content="nextdoor.com" property="og:site_name"/>
<meta content="114611681929998" property="fb:app_id"/>
<meta content="website" property="og:type"/>
<meta content="345600" property="og:ttl"/>
<meta content="https://d19rpgkrjeba2z.cloudfront.net/7e204fa0653577df/static/images/fb_share_logo2.png" property="og:image"/>
<meta content="800" property="og:image:width"/>
<meta content="450" property="og:image:height"/>
<meta content="The green Nextdoor house logo on a white background" property="og:image:alt"/>
<meta content="image/png" property="og:image:type"/>
<meta content="app-id=640360962" name="apple-itunes-app"/>
<meta content="
    Dublin, CA neighborhoods, events and more | Nextdoor
  " property="og:title"/>
<meta content="
  
      
          Dublin, CA has 20 neighborhoods including Dublin Ranch, West Dublin

In [11]:
soup.find_all('div',class_='column')[0]

<div class="column">
<div class="hood_group">
<p class="title">A</p>
<p><a href="https://nextdoor.com/neighborhood/arivare--dublin--ca/">Arivare</a></p>
<p><a href="https://nextdoor.com/neighborhood/arrivare--dublin--ca/">Arrivare</a></p>
</div>
<div class="hood_group">
<p class="title">C</p>
<p><a href="https://nextdoor.com/neighborhood/californiahighlands--dublin--ca/">California Highlands</a></p>
<p><a href="https://nextdoor.com/neighborhood/connollystation--dublin--ca/">Connolly Station</a></p>
</div>
<div class="hood_group">
<p class="title">D</p>
<p><a href="https://nextdoor.com/neighborhood/dublinranch--dublin--ca/">Dublin Ranch</a></p>
</div>
<div class="hood_group">
<p class="title">E</p>
<p><a href="https://nextdoor.com/neighborhood/echoparkdublin--dublin--ca/">Echo Park</a></p>
</div>
</div>

In [32]:
scrape_list=[]
for i in range(len(soup.find_all('div',class_='hood_group'))):
    scrape_list.append(soup.find_all('div',class_='hood_group')[i].findAll('p')[1:])
scrape_list

[[<p><a href="https://nextdoor.com/neighborhood/arivare--dublin--ca/">Arivare</a></p>,
  <p><a href="https://nextdoor.com/neighborhood/arrivare--dublin--ca/">Arrivare</a></p>],
 [<p><a href="https://nextdoor.com/neighborhood/californiahighlands--dublin--ca/">California Highlands</a></p>,
  <p><a href="https://nextdoor.com/neighborhood/connollystation--dublin--ca/">Connolly Station</a></p>],
 [<p><a href="https://nextdoor.com/neighborhood/dublinranch--dublin--ca/">Dublin Ranch</a></p>],
 [<p><a href="https://nextdoor.com/neighborhood/echoparkdublin--dublin--ca/">Echo Park</a></p>],
 [<p><a href="https://nextdoor.com/neighborhood/hansenranch--dublin--ca/">Hansen Ranch</a></p>,
  <p><a href="https://nextdoor.com/neighborhood/hillbrookbarnet--dublin--ca/">Hillbrook - Barnet</a></p>],
 [<p><a href="https://nextdoor.com/neighborhood/jordanranch--dublin--ca/">Jordan Ranch</a></p>],
 [<p><a href="https://nextdoor.com/neighborhood/longfordway--dublin--ca/">Longford Way</a></p>],
 [<p><a href="h

In [38]:
## function to flat a nested list with multiple levels of nesting 
## cannot quite understand

def removeNestings(l):

    for i in l:
        if type(i)==list:
            removeNestings(i)
        else:
            output.append(i)
    return output

In [40]:
output=[]
new_list=removeNestings(scrape_list)
new_list

[<p><a href="https://nextdoor.com/neighborhood/arivare--dublin--ca/">Arivare</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/arrivare--dublin--ca/">Arrivare</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/californiahighlands--dublin--ca/">California Highlands</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/connollystation--dublin--ca/">Connolly Station</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/dublinranch--dublin--ca/">Dublin Ranch</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/echoparkdublin--dublin--ca/">Echo Park</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/hansenranch--dublin--ca/">Hansen Ranch</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/hillbrookbarnet--dublin--ca/">Hillbrook - Barnet</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/jordanranch--dublin--ca/">Jordan Ranch</a></p>,
 <p><a href="https://nextdoor.com/neighborhood/longfordway--dublin--ca/">Longford Way</a></p>,
 <p><a href="https://nextdoor.co

In [43]:
neighborlist=[]
for row in new_list:
    neighborlist.append(row.text)
neighborlist

['Arivare',
 'Arrivare',
 'California Highlands',
 'Connolly Station',
 'Dublin Ranch',
 'Echo Park',
 'Hansen Ranch',
 'Hillbrook - Barnet',
 'Jordan Ranch',
 'Longford Way',
 'Piazza',
 'Positano',
 'Schaefer Ranch',
 'Tassajara Hills',
 'The Preserve',
 'Tralee',
 'Village/Dublin',
 'Village Pkwy',
 'Wallis Ranch',
 'West Dublin']

#### get coordinate of each neighborhood

In [46]:
# define a function to get coordinates
def get_coords(neighborhood):
    # initialize your variable to None
    lat_lng_coords=None
    #loop until you get the coordinates
    while(lat_lng_coords is None):
        g=geocoder.arcgis('{}, Dublin,CA'.format(neighborhood))
        lat_lng_coords=g.latlng
    return lat_lng_coords

In [48]:
dublin_coords=[get_coords(neighborhood) for neighborhood in neighborlist]
print(len(dublin_coords))
dublin_coords

20


[[37.70423000000005, -121.91634999999997],
 [37.70423000000005, -121.91634999999997],
 [37.51911000000007, -122.34159999999997],
 [53.35078000000004, -6.250399999999956],
 [37.71699815934966, -121.86834183008371],
 [34.07609000000008, -118.25580999999994],
 [37.703933078927726, -121.94460668615058],
 [37.72035506308477, -121.87480031230508],
 [37.713486706564254, -121.84724514671788],
 [37.723097765273735, -121.93200271749605],
 [37.693611000000004, -121.8915],
 [37.71418040682254, -121.85162745937807],
 [37.70395447807659, -121.98482738614437],
 [37.73092997907852, -121.87100180575341],
 [53.60818000000006, -6.18315999999993],
 [52.27031000000005, -9.698229999999967],
 [37.70603100000001, -121.92369299999999],
 [37.724765505032266, -121.92841234842534],
 [37.73385550938425, -121.87405999431675],
 [37.70556689919004, -121.87497577993891]]

In [51]:
# create dataframe to populate the coordinates into latitude and longitude
df_coords=pd.DataFrame(dublin_coords,columns=['Latitude','Longitude'])
df_coords['Neighborhood']=neighborlist
df_coords=df_coords[['Neighborhood','Latitude','Longitude']]
df_coords

,Neighborhood,Latitude,Longitude
0,Arivare,37.704230,-121.916350
1,Arrivare,37.704230,-121.916350
2,California Highlands,37.519110,-122.341600
3,Connolly Station,53.350780,-6.250400
4,Dublin Ranch,37.716998,-121.868342
5,Echo Park,34.076090,-118.255810
6,Hansen Ranch,37.703933,-121.944607
7,Hillbrook - Barnet,37.720355,-121.874800
8,Jordan Ranch,37.713487,-121.847245
9,Longford Way,37.723098,-121.932003


In [73]:
# data manipulation
nei_coords=df_coords.set_index('Neighborhood').drop_duplicates()
nei_coords['Neighborhood']=nei_coords.index.values
nei_coords=nei_coords.reset_index(drop=True)
nei_coords=nei_coords[['Neighborhood','Latitude','Longitude']]
#remove Dublin, OH and Dublin, Ireland
nei_coords=nei_coords[(nei_coords['Longitude']<=-121) & (nei_coords['Longitude']>=-122)].reset_index(drop=True)
nei_coords

,Neighborhood,Latitude,Longitude
0,Arivare,37.704230,-121.916350
1,Dublin Ranch,37.716998,-121.868342
2,Hansen Ranch,37.703933,-121.944607
3,Hillbrook - Barnet,37.720355,-121.874800
4,Jordan Ranch,37.713487,-121.847245
5,Longford Way,37.723098,-121.932003
6,Piazza,37.693611,-121.891500
7,Positano,37.714180,-121.851627
8,Schaefer Ranch,37.703954,-121.984827
9,Tassajara Hills,37.730930,-121.871002


#### create a map of Dublin with all neighborhoods superimposed on top

In [74]:
# get the coordinates of Dublin
address='Dublin, CA'
geolocator=Nominatim(user_agent='dublin_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Dublin, CA are {},{}'.format(latitude,longitude))

The geographical coordinate of Dublin, CA are 37.7021521,-121.9357918


In [75]:
# create map of Dublin using latitude and longitude values
map_dublin=folium.Map(location=[latitude,longitude],zoom_start=12)

#add markers to map
for lat, lng, neighborhood in zip(nei_coords['Latitude'], nei_coords['Longitude'], nei_coords['Neighborhood']):
    label='{}'.format(neighborhood)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7
    ).add_to(map_dublin)

map_dublin

### 3. Leverage Foursquare API to get venue data of Dublin neighborhoods

In [76]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


In [77]:
 # @hidden_cell

CLIENT_ID = 'GQGROWFIHLRKKB2RI15T00PYM4P0K25V3AL3D0O0GLPZLJVZ' # your Foursquare ID
CLIENT_SECRET = '5N0GX3GTBXP10BHF4ZOR2O4ANJWZS1J0H5VFDZKHJVXPMMDM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GQGROWFIHLRKKB2RI15T00PYM4P0K25V3AL3D0O0GLPZLJVZ
CLIENT_SECRET:5N0GX3GTBXP10BHF4ZOR2O4ANJWZS1J0H5VFDZKHJVXPMMDM


####  I will test to get 100 venues in my neighborhood (Arivare)

In [81]:
LIMIT=100
radius=1500

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nei_coords.loc[0,'Latitude'], 
    nei_coords.loc[0,'Longitude'], 
    radius, 
    LIMIT)
url # display UR

'https://api.foursquare.com/v2/venues/explore?&client_id=GQGROWFIHLRKKB2RI15T00PYM4P0K25V3AL3D0O0GLPZLJVZ&client_secret=5N0GX3GTBXP10BHF4ZOR2O4ANJWZS1J0H5VFDZKHJVXPMMDM&v=20180605&ll=37.70423000000005,-121.91634999999997&radius=1500&limit=100'

In [84]:
# send the GET request and examine the results
results=requests.get(url).json()
print(len(results))
nearby_venues=json_normalize(results['response']['groups'][0]['items'])
nearby_venues.head(10)

2


,reasons.count,reasons.items,referralId,venue.categories,venue.delivery.id,venue.delivery.provider.icon.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.name,venue.delivery.url,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ba83de4f964a520f0d239e3-0,"[{'id': '4bf58dd8d48988d101941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,4ba83de4f964a520f0d239e3,7100 Village Pkwy,US,Dublin,United States,NaN,140,"[7100 Village Pkwy, Dublin, CA 94568, United S...","[{'label': 'display', 'lat': 37.70537516433922...",37.705375,-121.915674,94568,CA,Combat Sports Academy / CSA CrossFit,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4a636498f964a520efc41fe3-1,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,4a636498f964a520efc41fe3,7090 Johnson Dr,US,Pleasanton,United States,NaN,542,"[7090 Johnson Dr, Pleasanton, CA 94588, United...","[{'label': 'display', 'lat': 37.69963637166717...",37.699636,-121.918394,94588,CA,ClubSport,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-53df184e498e3083a4e99662-2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,53df184e498e3083a4e99662,7050 Johnson Dr,US,Pleasanton,United States,NaN,428,"[7050 Johnson Dr, Pleasanton, CA 94588, United...","[{'label': 'display', 'lat': 37.70067002367042...",37.700670,-121.918193,94588,CA,DoubleTree by Hilton Hotel Pleasanton at the Club,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c3ccae4b169c9b6a2324668-3,"[{'id': '4bf58dd8d48988d1bc941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,4c3ccae4b169c9b6a2324668,7106 Dublin Blvd,US,Dublin,United States,NaN,542,"[7106 Dublin Blvd, Dublin, CA 94568, United St...","[{'label': 'display', 'lat': 37.70517193841891...",37.705172,-121.922392,94568,CA,Nothing Bundt Cakes,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b7847c9f964a520d1c12ee3-4,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,4b7847c9f964a520d1c12ee3,6999 Dublin Blvd,US,Dublin,United States,NaN,291,"[6999 Dublin Blvd, Dublin, CA 94568, United St...","[{'label': 'display', 'lat': 37.70548891931905...",37.705489,-121.919253,94568,CA,Athens Burgers,0,[],NaN
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cdc52933f6a8cfabeb0f0ea-5,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,4cdc52933f6a8cfabeb0f0ea,7112 Dublin Blvd,US,Dublin,United States,at Village Pkwy,559,"[7112 Dublin Blvd (at Village Pkwy), Dublin, C...","[{'label': 'display', 'lat': 37.70536077227197...",37.705361,-121.922545,94568,CA,Mr. Pickle's Sandwich Shop,0,[],NaN
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b495839f964a520ef6d26e3-6,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,4b495839f964a520ef6d26e3,6599 Dublin Blvd,US,Dublin,United States,at Dougherty Rd.,493,"[6599 Dublin Blvd (at Dougherty Rd.), Dublin, ...","[{'label': 'display', 'lat': 37.70459601282556...",37.704596,-121.910768,94568,CA,Yanagi Sushi & Grill,0,[],NaN
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b93edf7f964a5208b5a34e3-7,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,4b93edf7f964a5208b5a34e3,6058 Dougherty Rd,US,Dublin,United States,NaN,630,"[6058 Dougherty Rd, Dublin, CA 94568, United S...","[{'label': 'display', 'lat': 37.70760074515787...",37.707601,-121.910602,94568,CA,Denica's Real Food Kitchen,0,[],47774303
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5126db7fe4b067320fa7a50a-8,"[{'id': '5370f356bcbc57f1066c94

In [85]:
print('{} venues were retured by Foursquare'.format(nearby_venues.shape[0]))

100 venues were retured by Foursquare


In [87]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [88]:
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)

#filter columns
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

#filter the category for each row 
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)

nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Combat Sports Academy / CSA CrossFit,Martial Arts Dojo,37.705375,-121.915674
1,ClubSport,Gym,37.699636,-121.918394
2,DoubleTree by Hilton Hotel Pleasanton at the Club,Hotel,37.700670,-121.918193
3,Nothing Bundt Cakes,Cupcake Shop,37.705172,-121.922392
4,Athens Burgers,Burger Joint,37.705489,-121.919253
5,Mr. Pickle's Sandwich Shop,Sandwich Place,37.705361,-121.922545
6,Yanagi Sushi & Grill,Sushi Restaurant,37.704596,-121.910768
7,Denica's Real Food Kitchen,Bakery,37.707601,-121.910602
8,Caps & Taps,Beer Store,37.704620,-121.911608
9,Bay Club Pleasanton,Gym / Fitness Center,37.699534,-121.918729


In [89]:
nearby_venues.columns=[col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Combat Sports Academy / CSA CrossFit,Martial Arts Dojo,37.705375,-121.915674
1,ClubSport,Gym,37.699636,-121.918394
2,DoubleTree by Hilton Hotel Pleasanton at the Club,Hotel,37.700670,-121.918193
3,Nothing Bundt Cakes,Cupcake Shop,37.705172,-121.922392
4,Athens Burgers,Burger Joint,37.705489,-121.919253


#### Start exploring all neighborhoods

In [94]:
## function to repeat the same process for all the neighborhoods in Dublin, CA
def getNearbyVenues(names, latitudes, longitudes, radius=1200):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create API results URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        # make GET request
        results=requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood','Neighborhood Latitude','Neighborhood Longitude',
                           'Venue','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

In [95]:
dublin_venues = getNearbyVenues(names=nei_coords['Neighborhood'],
                                   latitudes=nei_coords['Latitude'],
                                   longitudes=nei_coords['Longitude']
                                  )


Arivare
Dublin Ranch
Hansen Ranch
Hillbrook - Barnet
Jordan Ranch
Longford Way
Piazza
Positano
Schaefer Ranch
Tassajara Hills
Village/Dublin
Village Pkwy
Wallis Ranch
West Dublin


In [97]:
dublin_venues.shape

(636, 7)

In [100]:
print('There are {} unique categories'.format(len(dublin_venues['Venue Category'].unique())))

There are 159 unique categories


### 4. Clustering analysis and Mapping